In [5]:
#Set up environment and import necessary dependencies

%matplotlib inline
from __future__ import division;
from __future__ import print_function;
from __future__ import absolute_import;
import sys, os;
from collections import OrderedDict, namedtuple, Counter;
import numpy as np;
np.random.seed(1234)

scriptsDir = os.environ.get("KERAS_DIR");
if (scriptsDir is None):
    raise Exception("Please set environment variable KERAS_DIR");
sys.path.insert(0,scriptsDir)
import keras

scriptsDir = os.environ.get("DEEPLIFT_DIR");
if (scriptsDir is None):
    raise Exception("Please set environment variable DEEPLIFT_DIR");
sys.path.insert(0,scriptsDir)

#Import some general util stuff
scriptsDir = os.environ.get("UTIL_SCRIPTS_DIR");
if (scriptsDir is None):
    raise Exception("Please set environment variable UTIL_SCRIPTS_DIR to point to av_scripts");
sys.path.insert(0,scriptsDir);
import pathSetter;
import util;
import fileProcessing as fp
from plottingUtilitiesPackage import matplotlibHelpers as mplh;

scriptsDir = os.environ.get("ENHANCER_SCRIPTS_DIR");
if (scriptsDir is None):
    raise Exception("Please set environment variable ENHANCER_SCRIPTS_DIR to point to enhancer_prediction_code");
sys.path.insert(0,scriptsDir+"/featureSelector/deepLIFFT/");
import criticalSubsetIdentification as csi
#import deepLIFT stuff
import deepLIFTutils
sys.path.insert(0,scriptsDir+"/featureSelector/deepLIFFT/kerasBasedBackprop");
from deepLIFTonGPU import ScoreTypes, Activations_enum, OutLayerInfo, getScoreFunc

In [6]:
#Load the keras model, make sure you normalise the weights
#of the first convolutional layer to be mean-centered at each position.
import vgg16_keras 
reload(vgg16_keras)
from vgg16_keras import *
import h5py
reload(deepLIFTutils)
#/srv/scratch/annashch/cs231n_project/vgg16_weights_dropout_regularization_augmenteddataTintContrast.stupidsgd.hdf5
keras_model=VGG_16('vgg16_weights_dropout_regularization_augmenteddataTintContrast.stupidsgd.hdf5')
#mean_normalise_softmax_weights(keras_model.layers[-2])


In [8]:
import keras_conversion as kc
reload(kc)
import deeplift_backend as B
reload(B)
import blobs
reload(blobs)
import models
reload(models)
import deeplift_util
reload(deeplift_util)
deeplift_model = kc.convert_sequential_model(keras_model)

Mean-normalising softmax


In [ ]:
target_contribs_func = deeplift_model.get_target_contribs_func(input_layer_idx=0)

In [ ]:
model.layers

In [ ]:
model.layers[0].get_config()

In [ ]:
import h5py
data=h5py.File('imagenet.validation.first50.hdf5','r')
validData=np.asarray(data['X_valid'])
Y_valid=np.asarray(data['Y_valid'])
data=validData 

In [ ]:
#get the outputs of the net on the validation set
reload(deepLIFTutils)
print(np.shape(data))
import theano
import deeplift_util

keras_output_func = theano.function([keras_model.layers[0].input],
                                     keras_model.layers[-1].get_output(train=False),
                                     allow_input_downcast=True)
keras_outputs = deeplift_util.run_function_in_batches(keras_output_func, input_data_list=[data])
keras_outputs=np.array(keras_outputs)

In [ ]:
deeplift_output_func = theano.function([deeplift_model.get_layers()[0].get_activation_vars()],
                                        deeplift_model.get_layers()[-1].get_activation_vars(),
                                        allow_input_downcast=True)
deeplift_outputs = deeplift_util.run_function_in_batches(deeplift_output_func, input_data_list=[data])
deeplift_outputs = np.array(deeplift_outputs)

In [ ]:
np.sum(np.abs(keras_outputs-deeplift_outputs))

In [ ]:
idx = 18
softmax_class = np.argmax(Y_valid[idx])
print("class:",softmax_class)

In [ ]:
#compile the gradient function
grad_var = theano.tensor.grad(
            theano.tensor.sum(model.layers[-2].get_output(train=False)[:,softmax_class]),
            model.layers[0].input)
grad_func = theano.function([model.layers[0].input], grad_var, allow_input_downcast=True)

In [ ]:
#compute the gradients
gradients = deeplift_util.run_function_in_batches(grad_func, input_data_list=[data])

In [ ]:
def rescale_func(values):
    values = 255.0*(values - np.min(values))/(np.max(values)-np.min(values))
    return values

In [ ]:
import skimage 
from skimage.io import imread
import matplotlib.pyplot as plt
img_original=imread('val_'+str(idx)+'.JPEG').astype(np.uint8)
plt.figure(figsize=(10, 2), facecolor='w')
plt.suptitle("")
plt.subplot(1, 5, 1)
plt.title('Original')
plt.imshow(img_original)
plt.subplot(1,5,2)
plt.title('Preprocessed')
plt.imshow(data[idx].transpose(1,2,0).astype(np.uint8))
plt.subplot(1,5,3)
plt.title('Gradient')
plt.imshow(rescale_func(gradients[idx]).transpose(1,2,0).astype(np.uint8))
plt.subplot(1,5,4)
plt.title('grad*inp (Taylor)')
plt.imshow(rescale_func(gradients[idx]*data[idx]).transpose(1,2,0).astype(np.uint8))
